In [1]:
import os
import re
import sys
import json
import time
from tqdm.notebook import tqdm
from datasets import load_dataset

/Users/sanghapark/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import torch
import random
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

## Load Data

In [3]:
paper_data = load_dataset('metamong1/summarization_paper', 
    use_auth_token='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')

Reusing dataset paper_summarization (/Users/sanghapark/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.1.0/4b4212164df52f676c62f61a7e782e1b53dbbf8c63447e0b69f1d4799ca18147)
100%|██████████| 2/2 [00:00<00:00, 93.23it/s]


In [4]:
paper_data

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 73640
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 18411
    })
})

In [5]:
train_data = paper_data['train']

## Extract Data

In [6]:
train_docs = [data['text'] for data in train_data]
train_titles = [data['title'] for data in train_data]

In [7]:
target_comp = re.compile('[a-zA-Zぁ-ゔァ-ヴー々〆〤一-龥]')

In [32]:
def check_title(title, comp) :
    org_chars = re.sub('\s+', '' , title) 
    target_chars = comp.findall(org_chars)

    char_rate = len(target_chars) / len(org_chars)

    return True if char_rate >= 0.3 else False

In [33]:
sep_titles = [title for title in tqdm(train_titles) if check_title(title, target_comp)]
print('Size of data which has to be checked : %d' %len(sep_titles))

  0%|          | 0/73640 [00:00<?, ?it/s]

Size of data which has to be checked : 11127


In [34]:
sep_titles[:10]

['불법 커뮤니티를 통한 비트코인 거래 추적 방법에 관한 연구 A Study of Bitcoin Transaction Tracking Method through Illegal Community',
 "장소로서의 동경(東京) - 1930년대 식민지 조선작가의 동경표상 Tokyo as Place - Tokyo representation of the colonial korean writers in 1930's",
 'PFM/PWM 듀얼 모드 피드백 기반 LED BLU 구동용 LLC 공진 변환 제어 IC 설계',
 'SOLAS 협약 개정안의 조기이행에 대한 연구 A Study on the Early Implementation of SOLAS Amendments',
 '沈宜麟의 『朝鮮童話大集』에 나타난 寓意의 유형과 그 의미 Allegory in the Great Collection of Fables from Joseon by Sim Eui-rin: Its Types and Meaning',
 '국제유가 변동이 주유소 휘발유 가격에 미치는 Rockets & Feathers 현상',
 '도연명시, 치유와 체험의 미학 Aesthetics of healing and experience in Tao yuanming literature',
 '탄소섬유강화 복합재료 성형시 화학수축에 의한 변형연구 Thermal Deformation of Carbon Fiber Reinforced Composite by Cure Shrinkage',
 '대한제국기 간도영토론의 등장과 종식 The Emergence and Cease of the Gando Territorial Claim in the Daehan Empire Period',
 'Random Effects Tobit 회귀모형을 이용한 교차로 교통사고 요인 분석 An Analysis on Vehicle Accident Factors of Intersections using Random Effects Tobit Regression Mode

## Selenium

In [19]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [20]:
chrome_path = '/Users/sanghapark/Desktop/Project/chromedriver'
driver = webdriver.Chrome(chrome_path)

<ipython-input-20-8d257ee82e31>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [21]:
title = '戰國楚地文獻 의문대체사 ‘害’,‘曷’의 의미고찰 -『清華簡 · 一』의 두 자형을 중심으로-'

In [30]:
driver.implicitly_wait(1.5)
driver.get('https://academic.naver.com/')

In [31]:
time.sleep(0.5) ## 0.5초

element = driver.find_element_by_name('query')
element.clear()

element.send_keys(title)
element.submit()

<ipython-input-31-d6b45ebb4143>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_name('query')


In [32]:
cur_url = driver.current_url

response = requests.get(cur_url)
bs = BeautifulSoup(response.content, 'html.parser')

In [33]:
div_info = bs.findAll('div', {'class' , 'ui_listing_info'})
div_info[0].find('a').text

'전국초지문헌 의문대체사 ‘해’,‘갈’의 의미고찰 -『청화간 · 일』의 두 자형을 중심으로- (戰國楚地文獻 의문대체사 ‘害’,‘曷’의 의미고찰 -『清華簡 ·...'

## Naver API

In [27]:
import re
import json
import urllib.request

In [39]:
def extract_title(content) :
    content = json.loads(content)
    title = content['items'][0]['title']

    title = re.sub('<b>', ' ', title)
    title = re.sub('</b>', ' ', title)
    title = re.sub('\s+' , ' ', title).strip()
    return title

In [29]:
client_id = "FcfC2Ls3KNm5z_Zd9OG_"
client_secret = "uV2UxplHq9"

In [44]:
title = '도연명시, 치유와 체험의 미학 Aesthetics of healing and experience in Tao yuanming literature'

In [45]:
encText = urllib.parse.quote(title)
url = "https://openapi.naver.com/v1/search/doc?query=" + encText # json 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if (rescode == 200) :
    response_body = response.read()
    response_content = response_body.decode('utf-8')

    org_name = extract_title(response_content)

In [46]:
org_name = extract_title(response_content)

In [47]:
org_name

'도연명시, 치유와 체험의 미학'